In [1]:
import sys
sys.path.insert(1, 'CMAPSS')
import eval_utils
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
params = {'legend.fontsize': 20,
          'figure.figsize': (9,6),
         'axes.labelsize': 20,
         'axes.titlesize':20,
         'xtick.labelsize':'xx-large',
         'axes.linewidth' : 2,
         'ytick.labelsize':'xx-large'}

plt.rcParams.update(params)

2024-05-01 11:13:39.356917: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import importlib
importlib.reload(eval_utils)

<module 'eval_utils' from '/baja/CMAPSS_GITHUB/eval_utils.py'>

In [3]:
# Load data partial window
PATH_IN = 'Data/N-CMAPSS'

# Load training data
X_windows = np.load(PATH_IN + '/X_windows.npy')
U_windows = np.load(PATH_IN + '/U_windows.npy')
C_windows = np.load(PATH_IN + '/C_windows.npy')
W_windows = np.load(PATH_IN + '/W_windows.npy')
Y_windows = np.load(PATH_IN + '/Y_windows.npy')

# Load testing data
X_windows_test = np.load(PATH_IN + '/X_windows_test.npy')
U_windows_test = np.load(PATH_IN + '/U_windows_test.npy')
C_windows_test = np.load(PATH_IN + '/C_windows_test.npy')
W_windows_test = np.load(PATH_IN + '/W_windows_test.npy')
Y_windows_test = np.load(PATH_IN + '/Y_windows_test.npy')

print(X_windows.shape, X_windows_test.shape)

(524644, 50, 14) (403697, 50, 14)


In [4]:
scaler_C = MinMaxScaler()
C_windows_scaled = scaler_C.fit_transform(C_windows.reshape(-1,1))
C_windows_test_scaled = scaler_C.transform(C_windows_test.reshape(-1,1))

In [5]:
C_windows_scaled = np.repeat(C_windows_scaled,X_windows.shape[1],axis = 1).reshape(-1,X_windows.shape[1],1)
C_windows_test_scaled = np.repeat(C_windows_test_scaled,X_windows_test.shape[1],axis = 1).reshape(-1,X_windows_test.shape[1],1)

In [6]:
# Ground Truth HI
PATH_IN = 'Data/N-CMAPSS'
SOURCE = 'N-CMAPSS_DS03'

ground_truth = pd.read_hdf(PATH_IN + "/" + SOURCE+'_cycle_test.h5', 'df')
ground_truth2 = pd.read_hdf(PATH_IN + "/" + SOURCE+'_cycle_dev.h5', 'df')

ground_truth = pd.concat([ground_truth,ground_truth2]).sort_values('unit').sort_index()


g_units = ground_truth.unit.values
g_his = ground_truth.HI.values

g_cycles = []
_, s_idx = np.unique(g_units, return_index=True)
for i in g_units[np.sort(s_idx)]:
    idx = np.ravel(g_units==i)
    g_cycles.extend(np.arange(idx.sum())+1)
g_cycles = np.array(g_cycles)

train_idx = np.ravel(np.isin(g_units,[np.unique(U_windows)]))
true_hi_train = g_his[train_idx].reshape(-1,1)
true_hi_cycles = g_cycles[train_idx]
true_hi_units = g_units[train_idx]


test_idx = np.ravel(np.isin(g_units,[np.unique(U_windows_test)]))
true_hi_test = g_his[test_idx].reshape(-1,1)
true_hi_cycles_test = g_cycles[test_idx]
true_hi_units_test = g_units[test_idx]
print(true_hi_train.shape, true_hi_test.shape)

(663, 1) (438, 1)


# Base Line RUL Model

In [7]:
def RUL_model(t=64,
              feature_X_in=14,
              feature_W_in=4,
              feature_H_in=1,
              feature_out_size=1,
              activation='relu',
              filter=[10,10,1],
              filter_size=10,
              useH=True):
    '''
    RUL_model: Generates a model for predicting Remaining Useful Life (RUL).

    Args:
        t (int): Time steps.
        feature_X_in (int): Number of features in input X.
        feature_W_in (int): Number of features in input W.
        feature_H_in (int): Number of features in input H.
        feature_out_size (int): Number of features in the output.
        activation (str): Activation function to use in the hidden layers.
        filter (list): List containing the number of filters for each convolutional layer.
        filter_size (int): Size of the convolutional filters.
        useH (bool): Flag indicating whether to include H as input.
                     If True, input will be [X, W, H, T].
                     If False, input will be [X, W, T].

    Returns:
        tf.keras.models.Model: RUL prediction model.

    Note:
        If useH is True, the model will accept input [X, W, H, T] and output Y.
        If useH is False, the model will accept input [X, W, T] and output Y.
    '''

    # Define input layers for X, W, T, and H
    x_in = tf.keras.layers.Input(shape=(t, feature_X_in), name="X_in")
    w_in = tf.keras.layers.Input(shape=(t, feature_W_in), name="W_in")
    t_in = tf.keras.layers.Input(shape=(t, 1), name="T_in")
    
    # Concatenate input data based on the useH flag
    if useH:
        h_in = tf.keras.layers.Input(shape=(t, feature_H_in), name="H_in")
        x = tf.keras.layers.Concatenate(axis=-1)([x_in, w_in, h_in, t_in])
    else:
        x = tf.keras.layers.Concatenate(axis=-1)([x_in, w_in, t_in])
      
    # Apply convolutional layers
    for i in filter:
        x = tf.keras.layers.Conv1D(i, filter_size, 1, padding='same', activation=activation)(x)

    # Flatten the output
    x = tf.keras.layers.Flatten()(x)
    
    # Apply dense layers
    y = tf.keras.layers.Dense(50, activation=activation)(x)
    y = tf.keras.layers.Dense(feature_out_size, activation='linear')(y)

    # Create the model with the appropriate inputs and outputs
    if useH:
        model = tf.keras.models.Model([x_in, w_in, t_in, h_in], y)
    else:
        model = tf.keras.models.Model([x_in, w_in, t_in], y)

    return model


In [8]:
useH=False
layers=  [10,10,1]
Window_size = X_windows.shape[1]
model = RUL_model(Window_size,filter = layers, useH = useH)

2024-05-01 11:13:44.565160: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-01 11:13:50.620088: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19914 MB memory:  -> device: 0, name: Tesla P40, pci bus id: 0000:3d:00.0, compute capability: 6.1
2024-05-01 11:13:50.622309: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22135 MB memory:  -> device: 1, name: Tesla P40, pci bus id: 0000:3e:00.0, compute capability: 6.1
2024-05-01 11:13:50.623803: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 14

In [9]:
result = eval_utils.train_and_evaluate_rul_model(X_windows, W_windows, C_windows_scaled, U_windows,Y_windows,_,X_windows_test, W_windows_test, C_windows_test_scaled, U_windows_test,Y_windows_test,_,
                                             model,useH = useH, runs = 1, epochs = 30, batch_size=248,learning_rate = 0.001,reset_weights = True)

RUN#:  0
reset_weights
Epoch 1/30


2024-05-01 11:13:59.931474: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2024-05-01 11:14:00.739667: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x4bd4d9b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-01 11:14:00.739774: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla P40, Compute Capability 6.1
2024-05-01 11:14:00.739802: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): Tesla P40, Compute Capability 6.1
2024-05-01 11:14:00.739826: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (2): Tesla P40, Compute Capability 6.1
2024-05-01 11:14:00.739848: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (3): Tesla P40, Compute Capability 6.1
2024-05-01 11:14:00.739870: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (4): Tesla P40, Compute Cap

 113/1904 [>.............................] - ETA: 18s - loss: 21.0789

KeyboardInterrupt: 

In [29]:
pd.DataFrame(result)

,MAE_TR,MAPE_TR,RMSE_TR,MAE_TS,MAPE_TS,RMSE_TS,TIME,I
0,23.688478,183.6308,29.074726,5.392739,22.195689,7.322837,179.745075,0


# HI included RUL model

## Example using ground truth

In [50]:
useH=True
layers=  [10,10,1]
Window_size = X_windows.shape[1]
model = RUL_model(Window_size,filter = layers, useH = useH)

In [47]:
hi_train = eval_utils.expand_hi(C_windows,U_windows,true_hi_train,true_hi_cycles.reshape(-1,1),true_hi_units.reshape(-1,1))     
hi_test = eval_utils.expand_hi(C_windows_test,U_windows_test,true_hi_test,true_hi_cycles_test,true_hi_units_test)

hi_train = np.repeat(hi_train,X_windows.shape[1],axis = 1).reshape(-1,X_windows.shape[1],1)
hi_test = np.repeat(hi_test,X_windows_test.shape[1],axis = 1).reshape(-1,X_windows_test.shape[1],1)

In [51]:
result = eval_utils.train_and_evaluate_rul_model(X_windows, W_windows, C_windows_scaled, U_windows,Y_windows,hi_train,X_windows_test, W_windows_test, C_windows_test_scaled, U_windows_test,Y_windows_test,hi_test,
                                             model,useH = useH, runs = 1, epochs = 30, batch_size=248,learning_rate = 0.001,reset_weights = True)

RUN#:  0
reset_weights
Epoch 1/30
1904/1904 [==============================] - 31s 14ms/step - loss: 5.5396 - val_loss: 4.5859
Epoch 2/30
1904/1904 [==============================] - 27s 13ms/step - loss: 4.5683 - val_loss: 4.5200
Epoch 3/30
1904/1904 [==============================] - 29s 14ms/step - loss: 4.4983 - val_loss: 4.5196
Epoch 4/30
1904/1904 [==============================] - 27s 13ms/step - loss: 4.4827 - val_loss: 4.4122
Epoch 5/30
1904/1904 [==============================] - 28s 13ms/step - loss: 4.4499 - val_loss: 4.3585
Epoch 6/30
1904/1904 [==============================] - 27s 13ms/step - loss: 4.4372 - val_loss: 4.5898
Epoch 7/30
1904/1904 [==============================] - 27s 13ms/step - loss: 4.4250 - val_loss: 4.3720
Epoch 8/30
1904/1904 [==============================] - 28s 13ms/step - loss: 4.3990 - val_loss: 4.4478
Epoch 9/30
1904/1904 [==============================] - 28s 13ms/step - loss: 4.3752 - val_loss: 4.5144
Epoch 10/30
1904/1904 [==================

In [52]:
pd.DataFrame(result)

,MAE_TR,MAPE_TR,RMSE_TR,MAE_TS,MAPE_TS,RMSE_TS,TIME,I
0,23.490616,185.796976,28.829237,4.533921,12.306307,6.74911,843.629539,0


## Example using supervised learning HI

In [10]:
useH=True
layers=  [10,10,1]
Window_size = X_windows.shape[1]
model = RUL_model(Window_size,filter = layers, useH = useH)

In [11]:
hi_train = np.load(PATH_IN + '/z_supervised_train.npy')
hi_test = np.load(PATH_IN + '/z_supervised_test.npy')

hi_train = eval_utils.expand_hi(C_windows,U_windows,hi_train,true_hi_cycles.reshape(-1,1),true_hi_units.reshape(-1,1))     
hi_test = eval_utils.expand_hi(C_windows_test,U_windows_test,hi_test,true_hi_cycles_test,true_hi_units_test)

hi_train = np.repeat(hi_train,X_windows.shape[1],axis = 1).reshape(-1,X_windows.shape[1],1)
hi_test = np.repeat(hi_test,X_windows_test.shape[1],axis = 1).reshape(-1,X_windows_test.shape[1],1)

/baja/CMAPSS_GITHUB/eval_utils.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.array(result)


In [12]:
result = eval_utils.train_and_evaluate_rul_model(X_windows, W_windows, C_windows_scaled, U_windows,Y_windows,hi_train,X_windows_test, W_windows_test, C_windows_test_scaled, U_windows_test,Y_windows_test,hi_test,
                                             model,useH = useH, runs = 1, epochs = 30, batch_size=248,learning_rate = 0.001,reset_weights = True)

RUN#:  0
reset_weights
Epoch 1/30
1904/1904 [==============================] - 28s 12ms/step - loss: 5.6186 - val_loss: 4.5920
Epoch 2/30
1904/1904 [==============================] - 26s 12ms/step - loss: 4.6581 - val_loss: 4.6364
Epoch 3/30
1904/1904 [==============================] - 24s 12ms/step - loss: 4.5819 - val_loss: 4.5412
Epoch 4/30
1904/1904 [==============================] - 25s 12ms/step - loss: 4.5189 - val_loss: 4.5558
Epoch 5/30
1904/1904 [==============================] - 25s 12ms/step - loss: 4.4833 - val_loss: 4.3938
Epoch 6/30
1904/1904 [==============================] - 25s 12ms/step - loss: 4.4383 - val_loss: 4.3377
Epoch 7/30
1904/1904 [==============================] - 24s 12ms/step - loss: 4.4057 - val_loss: 4.2717
Epoch 8/30
1904/1904 [==============================] - 25s 12ms/step - loss: 4.3761 - val_loss: 4.3618
Epoch 9/30
1904/1904 [==============================] - 25s 12ms/step - loss: 4.3594 - val_loss: 4.3117
Epoch 10/30
1904/1904 [==================

In [13]:
pd.DataFrame(result)

,MAE_TR,MAPE_TR,RMSE_TR,MAE_TS,MAPE_TS,RMSE_TS,TIME,I
0,23.806276,186.926377,29.214224,4.515249,15.789947,6.303746,756.772151,0


## Example using proposed constraint C HI

In [ ]:
useH=True
layers=  [10,10,1]
Window_size = X_windows.shape[1]
model = RUL_model(Window_size,filter = layers, useH = useH)

In [ ]:
hi_train = np.load(PATH_IN + '/z_constraint_c_train.npy')
hi_test = np.load(PATH_IN + '/z_constraint_c_test.npy')

hi_train = eval_utils.expand_hi(C_windows,U_windows,hi_train,true_hi_cycles.reshape(-1,1),true_hi_units.reshape(-1,1))     
hi_test = eval_utils.expand_hi(C_windows_test,U_windows_test,hi_test,true_hi_cycles_test,true_hi_units_test)

hi_train = np.repeat(hi_train,X_windows.shape[1],axis = 1).reshape(-1,X_windows.shape[1],1)
hi_test = np.repeat(hi_test,X_windows_test.shape[1],axis = 1).reshape(-1,X_windows_test.shape[1],1)

In [ ]:
result = eval_utils.train_and_evaluate_rul_model(X_windows, W_windows, C_windows_scaled, U_windows,Y_windows,hi_train,X_windows_test, W_windows_test, C_windows_test_scaled, U_windows_test,Y_windows_test,hi_test,
                                             model,useH = useH, runs = 1, epochs = 30, batch_size=248,learning_rate = 0.001,reset_weights = True)

In [ ]:
pd.DataFrame(result)